## Simulation for different routes generated when # of vehicles is changed for a constant demand.

In [1]:
#imports
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from latlong_vrp import append_list_as_row, update_file, makefile, plotgraph,update_fixed
import sys
import pandas as pd
import time
import math
import random

In [2]:
filepath = r'/Users/adityagoel/Documents/Thesis/Vrp-CurrentProgress/'
update_file(filepath)
sitepositions = pd.read_csv(''.join([filepath, 'site_position.csv']))
count   = len(sitepositions)
sapids = sitepositions['SAP_ID'].tolist()

In [3]:
def vehicle_demand_initialiser(count):
    vehicle_demands = []
    for i in range(count):
        vehicle_demands.append(random.randint(1, 10))
    
    with open('data_model_demand.txt', 'w') as f:
        for demand in vehicle_demands:
            f.write(str(demand) +'\n')
        f.close()

## Data Model

In [4]:
def create_data_model():
    """Stores the data for the problem."""
    
    
    matrix_data = pd.read_csv('newdata.csv')
    matrix = matrix_data.values.tolist()
    data = {}
    
    data['distance_matrix'] = []
    for distance_data_row in matrix:   
        data['distance_matrix'].append(distance_data_row)
    
    data['depot'] = 0
    data['num_vehicles'] = 10
    data['vehicle_capacities'] = [1] * data['num_vehicles']

#     Only needs to be done once
    vehicle_demand_initialiser(len(data['distance_matrix'][0]))

    data['demands'] = []
    with open('data_model_demand.txt', 'r') as f:
        for line in f:
            data['demands'].append((int(line)))
        f.close()
        
    return data    

## Printer Function

In [9]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        sap_index = []
        
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} -> '.format(node_index)
            sap_index.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {0} \n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        sap_index.append(manager.IndexToNode(index))
        print(plan_output)
        for z in sap_index:
            print(sapids[z],end=" -> ")
        print("\n")
        total_distance += route_distance
        total_load += route_load
        with open('output/{}_vehicle_result.txt'.format(data['num_vehicles']), 'a') as f:
            print(plan_output, file=f)
            for z in sap_index:
                print(sapids[z],end=" -> ",file=f)
            print("\n",file = f)

        max_route_distance = max(route_distance, max_route_distance)

    print('Maximum of the route distances: {}'.format(max_route_distance))    
    print('Total distance of all routes: {}'.format(total_distance))
    #print('Total load of all routes: {}'.format(total_load))


### Callbacks and finding solution

In [10]:
def vehicle_simulation_wrapper(data, number_of_vehicles, vehicle_capacity):
    
    
    # Maximum demand = 556 * 10 = 5560
    # Average demand = 556 * 5  = 2780
    
    # Capacity of Truck = vehicle_capacity
    # Min number of vehicle for solution = 2780/vehicle_capacity + 1
    
    '''UPDATE: DATA MODEL'''
    data['num_vehicles'] = number_of_vehicles
    data['vehicle_capacities'] = [vehicle_capacity] * number_of_vehicles
    
    '''CREATE MANAGER'''
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    
    '''CALLBACK FUNCTIONS -- DISTANCE AND DEMAND'''
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    '''CALLBACKS -- TRANSIT (DISTANCE) AND DEMAND'''
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    demand_callback_index  = routing.RegisterUnaryTransitCallback(demand_callback)

    '''ROUTING DIMENSION'''
    # Dimensions keep track of quantities that accumulate along a vehicle's route
    # Such as travel time or the total weight it is carrying.
    # If such a quantity exists in constraint (CVRP) or objective function (distance travelled)
    # Then we need to define a dimension for it 
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities - handles general case of varying capacity
        True,  # start cumulative to zero
        'Capacity')

    
    ''' OBJECTIVE FUNCTION'''
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    
    ''' SEARCH PARAMETERS'''
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)


    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)


    ''' PRINT TO FILE '''
    print("Solution with {} vehicles\n".format(number_of_vehicles))
    if solution:
        with open('output/{}_vehicle_routes'.format(number_of_vehicles), 'w') as f:
            f.write(" Solution with {} vehicles\n".format(number_of_vehicles))
            f.write('\n' + '-x-'*50 + '\n')



In [11]:
def main():
    # Instantiate the data problem.
    data = create_data_model()

    number_of_vehicles = [7, 10]
    #Capacity can be different for different vehicle,[100, 200, 500, 50, 750].
    #TODO: Change function definition and use args for vehicle capacity
    vehicle_capacity   = 1000 

    for number in number_of_vehicles:
        vehicle_simulation_wrapper(data, number, vehicle_capacity)
    
    


In [12]:
if __name__ == '__main__':
    main()

Solution with 7 vehicles

Solution with 10 vehicles



# Testing Things

In [21]:
# def create_data_model(count, max_num_of_vehicles):

def create_data_model():
    """Stores the data for the problem."""
    
    
    matrix_data = pd.read_csv('newdata.csv')
    matrix = matrix_data.values.tolist()
    data = {}
    
#     data['distance_matrix'] = []
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]

#     for distance_data_row in matrix:   
#         data['distance_matrix'].append(distance_data_row)
    
    data['depot'] = 0
    data['num_vehicles'] = 1
    data['vehicle_capacities'] = [100]

#     Only needs to be done once
    vehicle_demand_initialiser(len(data['distance_matrix']))

    data['demands'] = []
    with open('data_model_demand.txt', 'r') as f:
        for line in f:
            data['demands'].append((int(line)))
        f.close()
        
    return data

def update_data_model(data, number_of_vehicles):
    data['num_vehicles'] = number_of_vehicles
    data['vehicle_capacities'] = [500] * number_of_vehicles
    

In [22]:
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


In [23]:
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)


# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')




# Maximum vehicles in our simulation
number_of_vehicles = 5
update_data_model(data, number_of_vehicles)


In [24]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)


In [ ]:
# Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


In [97]:
data['demands']

[5,
 10,
 1,
 10,
 10,
 6,
 10,
 4,
 2,
 4,
 3,
 1,
 9,
 4,
 10,
 8,
 7,
 5,
 5,
 9,
 9,
 2,
 8,
 1,
 7,
 6,
 6,
 1,
 8,
 1,
 10,
 7,
 7,
 3,
 8,
 10,
 1,
 2,
 3,
 3,
 3,
 8,
 7,
 8,
 7,
 2,
 5,
 5,
 9,
 4,
 2,
 2,
 7,
 10,
 10,
 1,
 7,
 1,
 9,
 5,
 8,
 8,
 10,
 8,
 5,
 2,
 8,
 3,
 5,
 1,
 9,
 10,
 8,
 5,
 8,
 9,
 4,
 1,
 7,
 3,
 9,
 10,
 9,
 7,
 6,
 9,
 2,
 8,
 5,
 9,
 10,
 10,
 10,
 10,
 2,
 4,
 7,
 5,
 3,
 10,
 8,
 4,
 5,
 9,
 8,
 5,
 1,
 4,
 3,
 3,
 7,
 1,
 4,
 2,
 10,
 4,
 7,
 6,
 7,
 5,
 2,
 10,
 7,
 3,
 6,
 1,
 10,
 9,
 8,
 3,
 5,
 7,
 10,
 3,
 9,
 5,
 7,
 3,
 5,
 7,
 9,
 6,
 6,
 3,
 6,
 8,
 1,
 7,
 5,
 10,
 2,
 10,
 7,
 5,
 4,
 9,
 7,
 4,
 6,
 10,
 4,
 1,
 8,
 1,
 9,
 7,
 2,
 1,
 9,
 7,
 9,
 3,
 4,
 5,
 10,
 5,
 3,
 1,
 9,
 4,
 5,
 8,
 6,
 1,
 9,
 2,
 6,
 9,
 9,
 6,
 1,
 2,
 5,
 8,
 1,
 4,
 8,
 3,
 4,
 10,
 9,
 5,
 8,
 7,
 10,
 6,
 6,
 10,
 5,
 10,
 4,
 7,
 2,
 5,
 4,
 8,
 1,
 9,
 4,
 3,
 5,
 4,
 8,
 9,
 2,
 2,
 5,
 8,
 4,
 10,
 1,
 9,
 4,
 5,
 7,
 3,
 5,
 8,
 5,
 8,
 10,
 8,
 

In [53]:
len(data['distance_matrix'][0])

556

### Finding routes, iterating over number of vehicles

In [ ]:
number_of_vehicles = [5, 10, 15, 20]
for vehicles in number_of_vehicles
    data['num_vehicles'] = vehicles
#     do routing and figure out total cost
#     print solutions to file
#     
    continue
    